In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import urlparse


In [2]:
# url = "https://macxima.medium.com/python-extracting-domain-name-from-urls-using-regular-expressions-7fa2fc13088e"
url = "https://twitter.com/James_Kpatrick/status/320150923336892416/photo/1"

In [68]:
def get_title(url):
    try:
        soup = BeautifulSoup(urlopen(url))
        return soup.title.get_text()
    except:
        return "no_title"
    

def get_domain(url):
    domain = urlparse(url).netloc
    dot_split = domain.split(".")
    if len(dot_split) > 2:
        return ".".join(dot_split[1:])
    else:
        return domain

In [69]:
get_domain(url)

'twitter.com'

In [70]:
get_title(url)

'no_title'

# try on pandas

In [4]:
import sqlite3
import pandas as pd
import re


In [5]:
cnx = sqlite3.connect("../home.db")
df_tweets = pd.read_sql_query("SELECT id, full_text FROM tweets", cnx)

In [123]:
df_tweets

,id,full_text
0,70261648811761665,"I wish I had kept my 1,700 BTC @ $0.06 instead..."
1,303962112307363840,If corporate America built its own low orbit i...
2,320150923336892416,These two books contain the sum total of all h...
3,334391242395906048,I really do enjoy following @Aelkus.
4,380578576283611136,I'm just going to fill my van with dildos and ...
...,...,...
45433,1348928192728465409,Everything I hear about Q reminds me of the cr...
45434,1348928453471514624,RT @MaheshkarSaurav: Just earned the Gold Badg...
45435,1348929050828558341,RT @SimonGuy64: @norfolkhorns @JLewisStempel @...
45436,1348929175198048257,Everything I hear about Q reminds me of the 80...


In [7]:
urls = df_tweets[df_tweets["full_text"].str.contains("http")]

In [8]:
urls.head(10)

,id,full_text
2,320150923336892416,These two books contain the sum total of all h...
8,451159870306549761,Course slides from @Stanford and @stanfordsyms...
12,561051099423993856,"This is my favorite drone picture ever, https:..."
13,574518676575162369,Robert Capa's Omaha Beach by Dominique Bertail...
14,605933110958518272,Any assault on Mother Nature will be a suicide...
15,627427921624481792,600cals per day for 7 days reverses T2 diabete...
18,672595272875180032,"“She had, so to speak, personal control over r..."
19,674740046273056772,EAI680 analog computer readout is unique. Each...
20,681513454931345408,Found out there is a Fake Kanji creation conte...
22,697530278495977472,Great perspective and advice for those wanting...


In [10]:
def find_url(tweet):
    return re.findall("http\S+", tweet)

In [11]:
urls["urls"] = urls["full_text"].apply(find_url)

<ipython-input-11-6063dbd53349>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  urls["urls"] = urls["full_text"].apply(find_url)


In [12]:
urls.urls.str.len()

2        1
8        1
12       1
13       2
14       1
        ..
45431    1
45432    1
45433    1
45434    1
45436    1
Name: urls, Length: 26221, dtype: int64

In [13]:
urls["domains"] = urls.urls.apply(lambda x: [get_domain(d) for d in x])

<ipython-input-13-b15a2f58f598>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  urls["domains"] = urls.urls.apply(lambda x: [get_domain(d) for d in x])


In [14]:
urls.head(10)

,id,full_text,urls,domains
2,320150923336892416,These two books contain the sum total of all h...,[https://twitter.com/James_Kpatrick/status/320...,[twitter.com]
8,451159870306549761,Course slides from @Stanford and @stanfordsyms...,[http://stanford.edu/~zdar/week1.pdf],[stanford.edu]
12,561051099423993856,"This is my favorite drone picture ever, https:...",[https://twitter.com/grahamvsworld/status/5610...,[twitter.com]
13,574518676575162369,Robert Capa's Omaha Beach by Dominique Bertail...,[http://bandedessinee.blog.lemonde.fr/2014/06/...,"[blog.lemonde.fr, twitter.com]"
14,605933110958518272,Any assault on Mother Nature will be a suicide...,[https://twitter.com/wrathofgnon/status/605933...,[twitter.com]
15,627427921624481792,600cals per day for 7 days reverses T2 diabete...,[http://link.springer.com/article/10.1007/s001...,[springer.com]
18,672595272875180032,"“She had, so to speak, personal control over r...",[https://twitter.com/wrathofgnon/status/672595...,[twitter.com]
19,674740046273056772,EAI680 analog computer readout is unique. Each...,[https://twitter.com/qrs/status/67474004627305...,[twitter.com]
20,681513454931345408,Found out there is a Fake Kanji creation conte...,[http://www.sankeisquare.com/event/kanjicontes...,"[sankeisquare.com, twitter.com]"
22,697530278495977472,Great perspective and advice for those wanting...,[https://twitter.com/codevoyagers/status/69706...,[twitter.com]


In [22]:
flattened_domains_df = pd.DataFrame(
    [
        domain
        for domain_list in urls.domains
        for domain in domain_list
    ],
    columns=["domains"],
)

In [23]:
flattened_domains_df.head()

,domains
0,twitter.com
1,stanford.edu
2,twitter.com
3,blog.lemonde.fr
4,twitter.com


In [24]:
flattened_domains_df['domains'].unique().size

2624

In [25]:
flattened_domains_df['domains'].value_counts().head(20)

twitter.com              19648
arxiv.org                  647
techmeme.com               647
bit.ly                     533
github.com                 395
youtube.com                359
harvardartmuseums.org      344
youtu.be                   318
t.co                       253
medium.com                 164
nytimes.com                152
github.io                  143
circleboom.com             125
google.com                 124
bloomberg.com              120
substack.com                90
buff.ly                     89
techcrunch.com              88
                            85
wsj.com                     84
Name: domains, dtype: int64

# news sites

In [15]:
news_sites = pd.read_csv("news_sites.csv")

In [71]:
news_sites.HOSTNAME1.unique()

array(['1011now.com', '10news.com', '10tv.com', ..., 'zoiksonline.com',
       'zolmax.com', 'zumic.com'], dtype=object)

In [113]:
news_domains = news_sites.HOSTNAME1.unique().tolist()

In [114]:
len(news_domains)

9630

In [301]:
news_domains

['1011now.com',
 '10news.com',
 '10tv.com',
 '1150wima.com',
 '1190kex.com',
 '11alive.com',
 '123jump.com',
 '123macmini.com',
 '12newsnow.com',
 '1340wgau.com',
 '13abc.com',
 '13wham.com',
 '13wmaz.com',
 '148apps.com',
 '14news.com',
 '1500espn.com',
 '1600kush.com',
 '19actionnews.com',
 '1to1media.com',
 '1weddingsource.com',
 '225batonrouge.com',
 '24-7kpop.com',
 '247sports.com',
 '247wallst.com',
 '24dash.com',
 '24hourhiphop.com',
 '24hrs.ca',
 '24wrestling.com',
 '27east.com',
 '2dayfm.com.au',
 '2minutemedicine.com',
 '2paragraphs.com',
 '303magazine.com',
 '360nobs.com',
 '3blmedia.com',
 '3d-car-shows.com',
 '3dprint.com',
 '3g.co.uk',
 '3news.co.nz',
 '4029tv.com',
 '411mania.com',
 '41nbc.com',
 '4bc.com.au',
 '4flush.com',
 '4hoteliers.com',
 '4ni.co.uk',
 '560wgan.com',
 '570news.com',
 '570wkbn.com',
 '5newsonline.com',
 '610kvnu.com',
 '620wtmj.com',
 '630ched.com',
 '630wpro.com',
 '640toronto.com',
 '650keni.com',
 '660news.com',
 '680news.com',
 '6abc.com',
 '6la

In [115]:
for i in news_domains:
    if ".edu" in i:
        news_domains.remove(i)
len(news_domains)

9550

In [116]:
for i in news_domains:
    if ".edu" in i:
        print(i)

csun.edu
news.wisc.edu
uab.edu
ucf.edu
ucsd.edu
utexas.edu


In [117]:
for i in news_domains:
    if ".edu" in i:
        news_domains.remove(i)
len(news_domains)

9544

In [118]:
to_rem = [
    "youtube.com",df2[['team1','team2']] = pd.DataFrame(df2.teams.tolist()
    "co.ke",
    "co.nz",
    "co.tt",
    "co.tz",
    "co.ug",
    "co.uk",
    "co.za",
    "com.au",
    "com.lb",
    "com.ng",
    "com.ph",
    "com.pk",
    "com.sg",
    "com.ua",
    "linkedin.com",
]

In [119]:
news_domains = [x for x in news_domains if x not in to_rem]
len(news_domains)

9528

In [ ]:
news_domains =

In [305]:
urls.head(50)

,id,full_text,urls,domains
2,320150923336892416,None,[https://twitter.com/James_Kpatrick/status/320...,[twitter.com]
8,451159870306549761,None,[http://stanford.edu/~zdar/week1.pdf],[stanford.edu]
12,561051099423993856,None,[https://twitter.com/grahamvsworld/status/5610...,[twitter.com]
13,574518676575162369,None,[http://bandedessinee.blog.lemonde.fr/2014/06/...,"[blog.lemonde.fr, twitter.com]"
14,605933110958518272,None,[https://twitter.com/wrathofgnon/status/605933...,[twitter.com]
15,627427921624481792,None,[http://link.springer.com/article/10.1007/s001...,[springer.com]
18,672595272875180032,None,[https://twitter.com/wrathofgnon/status/672595...,[twitter.com]
19,674740046273056772,None,[https://twitter.com/qrs/status/67474004627305...,[twitter.com]
20,681513454931345408,None,[http://www.sankeisquare.com/event/kanjicontes...,"[sankeisquare.com, twitter.com]"
22,697530278495977472,None,[https://twitter.com/codevoyagers/status/69706...,[twitter.com]


In [121]:
filtered = flattened_domains_df[flattened_domains_df.domains.isin(news_domains)]

In [74]:
filtered['domains'].unique().size

298

In [75]:
filtered['domains'].unique().tolist()

['theguardian.com',
 'fbi.gov',
 'newscientist.com',
 'jamanetwork.com',
 'phys.org',
 'euractiv.com',
 'latimes.com',
 'city-journal.org',
 'scmp.com',
 'nymag.com',
 'infoq.com',
 'yahoo.com',
 'nature.com',
 'thehill.com',
 'theatlantic.com',
 'nationalgeographic.com',
 'europa.eu',
 'independent.co.uk',
 'ft.com',
 'nytimes.com',
 'bps.org.uk',
 'reuters.com',
 'washingtonpost.com',
 'bloomberg.com',
 'scientificamerican.com',
 'edsurge.com',
 'telegraph.co.uk',
 'outsideonline.com',
 'wired.com',
 'prospect.org',
 'cbc.ca',
 'cnn.com',
 'technologyreview.com',
 'theglobeandmail.com',
 'eurekalert.org',
 'techcrunch.com',
 'theonion.com',
 'plos.org',
 'prnewswire.com',
 'archdaily.com',
 'economist.com',
 'time.com',
 'crn.com',
 'abc.net.au',
 'hollywoodreporter.com',
 'ndtv.com',
 'krebsonsecurity.com',
 'politico.com',
 'thenation.com',
 'nbcnews.com',
 'edp24.co.uk',
 'engadget.com',
 'nypost.com',
 'wordpress.com',
 'wsj.com',
 'psychiatryonline.org',
 'oup.com',
 'france24.c

# filter original df

In [268]:
df_tweetzz = df_tweets.copy()
df_tweetzz.head()

,id,full_text
0,70261648811761665,"I wish I had kept my 1,700 BTC @ $0.06 instead..."
1,303962112307363840,If corporate America built its own low orbit i...
2,320150923336892416,These two books contain the sum total of all h...
3,334391242395906048,I really do enjoy following @Aelkus.
4,380578576283611136,I'm just going to fill my van with dildos and ...


remove from domains:
- twitter.com
- bit.ly
- youtu.be                 
- t.co
- buff.ly    

In [269]:
def remove_links(tweet):
    tweet = re.sub(r'bit.ly/\S+', '', tweet)
    tweet = re.sub(r't.co/\S+', '', tweet)
    tweet = re.sub(r'buff.ly/\S+', '', tweet)
    tweet = re.sub(r'twitter.com/\S+', '', tweet)
    return tweet

In [270]:
def remove_empty_str(l):
    for i in l:
        if len(i) == 0:
            l.remove(i)
    return(l)

In [271]:
df_tweetzz["urls"] = df_tweetzz["full_text"].apply(find_url)
df_tweetzz["urls"] = df_tweetzz.urls.apply(lambda x: [remove_links(d) for d in x])
df_tweetzz["domains"] = df_tweetzz.urls.apply(lambda x: [get_domain(d) for d in x])
df_tweetzz["domains"] = df_tweetzz.domains.apply(remove_empty_str)
df_tweetzz.drop(["urls"], axis=1, inplace=True)
df_tweetzz["len"]= df_tweetzz.domains.str.len()
df_tweetzz.head(15)

,id,full_text,domains,len
0,70261648811761665,"I wish I had kept my 1,700 BTC @ $0.06 instead...",[],0
1,303962112307363840,If corporate America built its own low orbit i...,[],0
2,320150923336892416,These two books contain the sum total of all h...,[],0
3,334391242395906048,I really do enjoy following @Aelkus.,[],0
4,380578576283611136,I'm just going to fill my van with dildos and ...,[],0
5,403659186791133184,"The advice I wish I'd received was ""keep progr...",[],0
6,438264421128089600,The Bitcoin lottery winning letters have just ...,[],0
7,448923765779755009,"""You have enemies? Good. That means you've sto...",[],0
8,451159870306549761,Course slides from @Stanford and @stanfordsyms...,[stanford.edu],1
9,478087637031325697,60% of my fav links from 10 yrs ago are 404. I...,[],0


In [272]:
new_columns_list = []
for i in range(df_tweetzz.len.max()):
    new_columns_list.append(f"domain{i+1}")

In [273]:
new_columns_list

['domain1', 'domain2', 'domain3', 'domain4', 'domain5', 'domain6']

In [274]:
df_tweetzz[new_columns_list] = pd.DataFrame(df_tweetzz.domains.tolist())

In [276]:
df_tweetzz.head(15)

,id,full_text,domains,len,domain1,domain2,domain3,domain4,domain5,domain6
0,70261648811761665,"I wish I had kept my 1,700 BTC @ $0.06 instead...",[],0,None,None,None,None,None,None
1,303962112307363840,If corporate America built its own low orbit i...,[],0,None,None,None,None,None,None
2,320150923336892416,These two books contain the sum total of all h...,[],0,None,None,None,None,None,None
3,334391242395906048,I really do enjoy following @Aelkus.,[],0,None,None,None,None,None,None
4,380578576283611136,I'm just going to fill my van with dildos and ...,[],0,None,None,None,None,None,None
5,403659186791133184,"The advice I wish I'd received was ""keep progr...",[],0,None,None,None,None,None,None
6,438264421128089600,The Bitcoin lottery winning letters have just ...,[],0,None,None,None,None,None,None
7,448923765779755009,"""You have enemies? Good. That means you've sto...",[],0,None,None,None,None,None,None
8,451159870306549761,Course slides from @Stanford and @stanfordsyms...,[stanford.edu],1,stanford.edu,None,None,None,None,None
9,478087637031325697,60% of my fav links from 10 yrs ago are 404. I...,[],0,None,None,None,None,None,None


#filter out domains with news

df[feature] = (df["number_of_doors"] == v).astype(int)

0        0
1        0
2        0
3        0
4        0
        ..
45433    0
45434    0
45435    0
45436    0
45437    0
Name: domain1, Length: 45438, dtype: int64

In [284]:
new_columns_list

['domain1', 'domain2', 'domain3', 'domain4', 'domain5', 'domain6']

In [286]:
for v in new_columns_list:
    df_tweetzz[v] = df_tweetzz[v].isin(news_domains).astype(int)

In [287]:
df_tweetzz.head(15)

,id,full_text,domains,len,domain1,domain2,domain3,domain4,domain5,domain6
0,70261648811761665,"I wish I had kept my 1,700 BTC @ $0.06 instead...",[],0,0,0,0,0,0,0
1,303962112307363840,If corporate America built its own low orbit i...,[],0,0,0,0,0,0,0
2,320150923336892416,These two books contain the sum total of all h...,[],0,0,0,0,0,0,0
3,334391242395906048,I really do enjoy following @Aelkus.,[],0,0,0,0,0,0,0
4,380578576283611136,I'm just going to fill my van with dildos and ...,[],0,0,0,0,0,0,0
5,403659186791133184,"The advice I wish I'd received was ""keep progr...",[],0,0,0,0,0,0,0
6,438264421128089600,The Bitcoin lottery winning letters have just ...,[],0,0,0,0,0,0,0
7,448923765779755009,"""You have enemies? Good. That means you've sto...",[],0,0,0,0,0,0,0
8,451159870306549761,Course slides from @Stanford and @stanfordsyms...,[stanford.edu],1,0,0,0,0,0,0
9,478087637031325697,60% of my fav links from 10 yrs ago are 404. I...,[],0,0,0,0,0,0,0


In [297]:
df_tweetzz["is_news"] = df_tweetzz[new_columns_list].sum(axis=1)
df_tweetzz.drop(new_columns_list, axis=1, inplace=True)

In [300]:
df_tweetzz[df_tweetzz["is_news"] > 0]

,id,full_text,domains,len,is_news
88,986941808348409856,My comments in The Guardian on implementation ...,[theguardian.com],1,1
165,1121030235108118528,@wmarybeard @Lord_Keynes2 @Steve_Sailer The Yo...,[fbi.gov],1,1
173,1129155364404858880,Peasants in medieval England ate a diet of mea...,[newscientist.com],1,1
197,1153145699086680069,The Finns know the score: regular sauna usage ...,[jamanetwork.com],1,1
201,1158201768305496070,The West is excellent at exporting their their...,[phys.org],1,1
...,...,...,...,...,...
45243,1348857700176723969,Indonesia-based BukuKas raises $10M Series A l...,"[techcrunch.com, techmeme.com]",2,1
45310,1348882982573113346,"*Hmmm, interesting sidelight on the late Ashli...",[nymag.com],1,1
45346,1348894052750483456,"Curve, which combines all of a customer's debi...","[cnbc.com, techmeme.com]",2,1
45394,1348906736095293442,https://www.sciencemag.org/news/2021/01/us-law...,[sciencemag.org],1,1


In [63]:
import tweepy
from IPython.core.display import display, HTML

auth = json.load(open("../auth/auth.json"))
auth = tweepy.OAuthHandler(auth["api_key"], auth["api_secret_key"])
api = tweepy.API(auth, wait_on_rate_limit=True)

def display_tweet(tweet_id):
    et = api.get_oembed(tweet_id)["html"]
    return display(HTML(et))

In [246]:
display_tweet(70261648811761665)